In [17]:
!pip install keras

# Import Libraries - Pandas, OS, Tensorflow, etc.
import pandas as pd
import os
import shutil 
import os.path
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

import numpy as np
from sklearn.utils.multiclass import unique_labels
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Sequential
from keras.applications import VGG19 #For Transfer Learning
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from keras.utils import to_categorical
from pathlib import Path


In [18]:
# path, dirs, files = next(os.walk("./data/labeled"))
# file_count = len(files)
# print("Number of files: " + str(file_count))

# train_set = len(files) * 0.6
# valid_set = len(files) * 0.4

# train_count = 0
# validation_count = 0

# overall_count = 0
# for file in os.listdir("./data/labeled"):
#     if (file != ".ipynb_checkpoints"):
#         if (overall_count < train_set):
#             source = './data/labeled/' + str(file)
#             if (file.endswith('0.jpg')):
#                 destination = './train/non-melanoma/' + str(file)
#             else:
#                 destination = './train/melanoma/' + str(file)
                
#             shutil.copy (source, destination)
            
# #             print("Train: " + str(file))
#             train_count += 1
#         else:
#             source1 = './data/labeled/' + str(file)
#             if (file.endswith('0.jpg')):
#                 destination1 = './valid/non-melanoma/' + str(file)
#             else:
#                 destination1 = './valid/melanoma/' + str(file)

#             shutil.copy (source1, destination1)
# #             print("Validation: " + str(file))
#             validation_count += 1
#     overall_count += 1

# print(train_count)
# print(validation_count)
# print(overall_count)

# path, dirs, files = next(os.walk("./valid/melanoma"))
# file_count = len(files)
# print ("Validation dataset length: " + str(file_count))

# path, dirs, files = next(os.walk("./train"))
# file_count = len(files)
# print("Train dataset length: " + str(file_count))

In [19]:
# shutil.rmtree('./train')
# shutil.rmtree('./valid')

# !mkdir train
# !mkdir valid
# !mkdir train/melanoma
# !mkdir train/non-melanoma
# !mkdir valid/melanoma
# !mkdir valid/non-melanoma

In [20]:
# Loading in training data
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [21]:
train_generator = train_datagen.flow_from_directory(
        './train_processed',
        target_size=(32, 32),
        batch_size=16,
        class_mode='categorical')

Found 720 images belonging to 2 classes.


In [22]:
# Loading in testing data
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = train_datagen.flow_from_directory(
        './valid_processed',
        target_size=(32, 32),
        batch_size=16,
        class_mode='categorical')

Found 480 images belonging to 2 classes.


In [23]:
# initialising sequential model and adding layers to it
# cnn = tf.keras.models.Sequential()
# cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, activation='relu', input_shape=(3, 350, 350)))
# cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, activation='relu'))
# cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
# cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# cnn.add(tf.keras.layers.Flatten())
# cnn.add(tf.keras.layers.Dense(128, activation='relu'))
# cnn.add(tf.keras.layers.Dense(64, activation='relu'))
# cnn.add(tf.keras.layers.Dense(8, activation='softmax'))

# Initializing sequential model that makes use of convolutional, pooling, and dense layers
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=5, kernel_size=3, activation='relu', input_shape=(32, 32, 3), padding='same'))
# #cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# cnn.add(tf.keras.layers.MaxPool2D((2,2), strides=(2,2), padding='same'))
# cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, activation='relu', padding = 'same'))
# cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding = 'same'))
# cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding = 'same'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='same'))
cnn.add(tf.keras.layers.Flatten())
# cnn.add(tf.keras.layers.Dense(128, activation='relu'))
cnn.add(tf.keras.layers.Dense(64, activation='relu'))
cnn.add(tf.keras.layers.Dense(2, activation='softmax'))

In [24]:
# config = tf.ConfigProto(
#         device_count = {'GPU': 0}
#     )
# sess = tf.Session(config=config)

In [25]:
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 5)         140       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 5)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                81984     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
Total params: 82,254
Trainable params: 82,254
Non-trainable params: 0
_________________________________________________________________


In [26]:
# finally compile and train the cnn
cnn.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
cnn.fit(x=train_generator, validation_data=test_generator, epochs=10)

Epoch 1/10
45/45 [==============================] - 3s 64ms/step - loss: 0.6703 - accuracy: 0.5917 - val_loss: 0.6793 - val_accuracy: 0.5750
Epoch 2/10
45/45 [==============================] - 2s 40ms/step - loss: 0.6506 - accuracy: 0.6139 - val_loss: 0.5965 - val_accuracy: 0.6438
Epoch 3/10
45/45 [==============================] - 2s 42ms/step - loss: 0.5860 - accuracy: 0.7125 - val_loss: 0.5864 - val_accuracy: 0.6812
Epoch 4/10
45/45 [==============================] - 3s 62ms/step - loss: 0.5593 - accuracy: 0.7236 - val_loss: 0.5830 - val_accuracy: 0.6792
Epoch 5/10
45/45 [==============================] - 2s 51ms/step - loss: 0.5271 - accuracy: 0.7431 - val_loss: 0.6371 - val_accuracy: 0.6625
Epoch 6/10
45/45 [==============================] - 2s 45ms/step - loss: 0.4942 - accuracy: 0.7917 - val_loss: 0.6351 - val_accuracy: 0.6583
Epoch 7/10
45/45 [==============================] - 2s 37ms/step - loss: 0.5189 - accuracy: 0.7333 - val_loss: 0.5864 - val_accuracy: 0.6646
Epoch 8/10
45

In [27]:
# !mkdir model
cnn.save('model/my_model')

INFO:tensorflow:Assets written to: model/my_model/assets


In [28]:
import cv2
import tensorflow as tf

CATEGORIES = ["Dog", "Cat"]  # will use this to convert prediction num to string value

def prepare(filepath):
    IMG_SIZE = 32  # 50 in txt-based
    img_array = cv2.imread(filepath)  # read in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)  # return the image with shaping that TF wants.

path, dirs, files = next(os.walk("./data/test"))
file_count = len(files)

predictions = []
final = []
count = 0
for file in os.listdir('./data/test'):
    print(file)
    if (file != ".ipynb_checkpoints"):
        prediction = cnn.predict([prepare('data/test/' + str(file))]) 
        if (prediction[0][0] > prediction[0][1]):
            predicted_result = "non-melanoma"
            if (file.endswith('0.jpg')):
                final_result = "non-melanoma"
            else:
                final_result = "melanoma"
        else:
            predicted_result = "melanoma"
            if (file.endswith('0.jpg')):
                final_result = "non-melanoma"
            else:
                final_result = "melanoma"

        if (predicted_result == final_result):
            count += 1
        predictions.append(predicted_result)
        final.append(final_result)
    
print(count)
print(file_count-1)
    

555_0.jpg
510_0.jpg
219_1.jpg
398_0.jpg
320_0.jpg
118_1.jpg
365_0.jpg
448_0.jpg
160_1.jpg
358_0.jpg
125_1.jpg
430_0.jpg
77_1.jpg
32_1.jpg
475_0.jpg
488_0.jpg
261_1.jpg
224_1.jpg
595_0.jpg
568_0.jpg
381_0.jpg
245_1.jpg
200_1.jpg
414_0.jpg
53_1.jpg
16_1.jpg
451_0.jpg
144_1.jpg
101_1.jpg
339_0.jpg
509_0.jpg
8_1.jpg
571_0.jpg
534_0.jpg
285_1.jpg
429_0.jpg
304_0.jpg
341_0.jpg
179_1.jpg
184_1.jpg
93_1.jpg
278_1.jpg
491_0.jpg
573_0.jpg
536_0.jpg
91_1.jpg
493_0.jpg
186_1.jpg
306_0.jpg
343_0.jpg
287_1.jpg
29_1.jpg
146_1.jpg
103_1.jpg
416_0.jpg
51_1.jpg
14_1.jpg
453_0.jpg
247_1.jpg
202_1.jpg
383_0.jpg
263_1.jpg
88_1.jpg
226_1.jpg
432_0.jpg
75_1.jpg
30_1.jpg
477_0.jpg
162_1.jpg
127_1.jpg
597_0.jpg
557_0.jpg
512_0.jpg
48_1.jpg
322_0.jpg
367_0.jpg
593_0.jpg
71_1.jpg
436_0.jpg
473_0.jpg
34_1.jpg
166_1.jpg
123_1.jpg
267_1.jpg
222_1.jpg
326_0.jpg
363_0.jpg
553_0.jpg
516_0.jpg
302_0.jpg
347_0.jpg
283_1.jpg
68_1.jpg
95_1.jpg
497_0.jpg
182_1.jpg
577_0.jpg
532_0.jpg
243_1.jpg
206_1.jpg
387_0.jpg
142_1.jpg

In [29]:
print(predictions)

['non-melanoma', 'non-melanoma', 'melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'melanoma', 'non-melanoma', 'non-melanoma', 'melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'melanoma', 'non-melanoma', 'melanoma', 'melanoma', 'non-melanoma', 'melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'melanoma', 'non-melanoma', 'non-melanoma', 'melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma', 'non-melanoma',

In [30]:
print(prediction)

[[1. 0.]]


In [15]:
new_model = tf.keras.models.load_model('./model/my_model')

In [16]:
unique_count = {'anger' : 0,
 'surprise' : 0,
 'disgust' : 0,
 'fear' : 0,
 'neutral' : 0,
 'happiness' : 0,
 'sadness' : 0,
 'contempt' : 0}
for test_file in os.listdir("./test_resize"):
    #print(test_file)
    prediction = new_model.predict([prepare("./test_resize/" + test_file)])
    prediction_list = prediction.tolist()
    prediction_list = prediction_list[0]
    max_value = max(prediction_list)
    maximum = -1
    for index in prediction_list:
        if (maximum < index):
            maximum = index
    unique_emotion = unique_emotions[prediction_list.index(maximum)]
    
    unique_count[unique_emotion] += 1

for key in unique_count.keys():
    print(key + ":" + str(unique_count[key]))
    

FileNotFoundError: [Errno 2] No such file or directory: './test_resize'

In [ ]:
unique_pictures = {'anger' : [],
 'surprise' : [],
 'disgust' : [],
 'fear' : [],
 'neutral' : [],
 'happiness' : [],
 'sadness' : [],
 'contempt' : []}

for test_file in os.listdir("./test_resize"):
    #print(test_file)
    prediction = new_model.predict([prepare("./test_resize/" + test_file)])
    prediction_list = prediction.tolist()
    prediction_list = prediction_list[0]
    max_value = max(prediction_list)
    maximum = -1
    for index in prediction_list:
        if (maximum < index):
            maximum = index
    unique_emotion = unique_emotions[prediction_list.index(maximum)]
    
    unique_pictures[unique_emotion].append("./test_resize/" + test_file)

In [ ]:
plt.imshow(cv2.imread(unique_pictures['sadness'][8]))

In [ ]:
val = cv2.imread(unique_pictures['anger'][0])
print(val)